In [173]:
import pandas as pd

%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [174]:
df = pd.read_csv("purchase_data.csv")

In [175]:
# player count

total_players = pd.DataFrame({df.SN.nunique()})
total_players.columns = ["Total Players"]
total_players

,Total Players
0,576


In [176]:
# purchasing Analysis (Total)

purchasing_analysis = pd.DataFrame(
    {
        "Number of Unique Items": [df["Item ID"].nunique()],
        "Average Price": "$" + format(df["Price"].mean(), ".2f"),
        "Number of Purchases": [df["SN"].count()],
        "Total Revenue": "$" + format(df["Price"].sum(), ".2f"),
    }
)

purchasing_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [191]:
# Gender Demographics - INCOMPLETE

# Column index 0
g_count = df.groupby(df["Gender"])["SN"].nunique().sort_values(ascending=False)


# Column Index 1
total_count = df["SN"].nunique()


def gender_per(e):
    return e / total_count


percent_of_players = [gender_per(e) for e in g_count]

gender_dem = pd.DataFrame(
    {"Total Count": g_count, "Percent of Players": percent_of_players}
)
gender_dem

# Display in new DF

,Total Count,Percent of Players
Gender,,
Male,484,0.840278
Female,81,0.140625
Other / Non-Disclosed,11,0.019097


In [215]:
# Purchase Analysis (GENDER)

purchase_count = df.groupby(df["Gender"])["Purchase ID"].count()

g_avg_price = df.groupby(df["Gender"])["Price"].mean()

g_total_purchase = df.groupby(df["Gender"])["Price"].sum()

g_avg_total_purchase = (g_total_purchase / g_count).map("${:,.2f}".format)


gen_pur = pd.DataFrame(
    {
        "Purchase Count": purchase_count,
        "Average Purchase Price": g_avg_price,
        "Total Purchase Value": g_total_purchase,
        "Avg Total Purchase per Person": g_avg_total_purchase,
    }
)

gen_pur

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.203009,361.94,$4.47
Male,652,3.017853,1967.64,$4.07
Other / Non-Disclosed,15,3.346000,50.19,$4.56


In [202]:
# Age Demographics
# broken

df_unique = df[["Age", "SN"]].drop_duplicates()

df_unique["age_counts"] = pd.cut(
    df["Age"],
    bins=[0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 200],
    labels=["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"],
)

age_count = df_unique["age_counts"].value_counts()
age_count

20-24    258
15-19    107
25-29     77
30-34     52
35-39     31
10-14     22
<10       17
40+       12
Name: age_counts, dtype: int64